Project Report Week5 Final Report of the Capstone Project by Anand Sundaresan

Lets begin with the identification of the Suburbs of New Zealand Wiki Page

Steps involved would be to:
a) Extract neigbourhood content
b) Identification of Geographical co-ordinates
c) Obtain Venue data using Foursquare API
d) Clustering within the data
e) Identify among Clusters which is the best place in NZ  to open a shopping mall

In [26]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json 
from geopy.geocoders import Nominatim 

import requests 
from bs4 import BeautifulSoup 

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib for plotting
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means for clustering data
from sklearn.cluster import KMeans

import folium 
print("Libraries imported.")

Libraries imported.


In [25]:
!conda install -c conda-forge geopy -y

Solving environment: done

# All requested packages already installed.



In [15]:
!conda install -c conda-forge folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         112 KB

The following NEW packages will be INSTALLED:

    branca:  0.3.1-py_0  conda-forge
    folium:  0.10.1-py_0 conda-forge
    vincent: 0.4.4-py_1  conda-forge


branca-0.3.1         | 25 KB     | ##################################### | 100% 
vincent-0.4.4        | 28 KB     | ##################################### | 100% 
folium-0.10.1        | 59 KB   

In [17]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_New_Zealand").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,► Suburbs of Auckland‎ (208 P)
1,"► Suburbs of Blenheim, New Zealand‎ (5 P)"
2,"► Suburbs of Christchurch‎ (2 C, 73 P)"
3,"► Suburbs of Dunedin‎ (2 C, 49 P)"
4,"► Suburbs of Gisborne, New Zealand‎ (11 P)"


In [18]:
kl_df.shape

(19, 1)

In [40]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Suburbs of Auckland, New Zealand'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [41]:
coords = [get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [35]:
!pip install geocoder

     |████████████████████████████████| 102kB 5.6MB/s ta 0:00:011


In [39]:
import geocoder

In [42]:
coords

[[-36.70918289999997, 174.73309173400003],
 [-36.91049195011247, 174.92694158581452],
 [-36.85760256310664, 174.7682113019704],
 [-36.844479, 174.748203],
 [-36.916979880283165, 174.70229688363324],
 [-36.84839904099994, 174.76438785300002],
 [-36.84839904099994, 174.76438785300002],
 [-36.916979880283165, 174.70229688363324],
 [-36.87178247176209, 174.6387239274212],
 [-36.846656646065895, 174.63014997072523],
 [-36.90892587892125, 174.917731510103],
 [-36.8129778449448, 174.78683320155363],
 [-40.349207810999985, 175.61196363500005],
 [-36.89080249113294, 174.76489125921157],
 [-36.91049195011247, 174.92694158581452],
 [-36.926109, 174.681216],
 [-36.906813, 174.54694949999998],
 [-36.893583, 174.93007950000003],
 [-36.87178247176209, 174.6387239274212]]

In [43]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [44]:
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [45]:
print(kl_df.shape)
kl_df

(19, 3)


,Neighborhood,Latitude,Longitude
0,► Suburbs of Auckland‎ (208 P),-36.709183,174.733092
1,"► Suburbs of Blenheim, New Zealand‎ (5 P)",-36.910492,174.926942
2,"► Suburbs of Christchurch‎ (2 C, 73 P)",-36.857603,174.768211
3,"► Suburbs of Dunedin‎ (2 C, 49 P)",-36.844479,174.748203
4,"► Suburbs of Gisborne, New Zealand‎ (11 P)",-36.916980,174.702297
5,► Suburbs of Greymouth‎ (3 P),-36.848399,174.764388
6,"► Suburbs of Hamilton, New Zealand‎ (50 P)",-36.848399,174.764388
7,"► Suburbs of Hastings, New Zealand‎ (11 P)",-36.916980,174.702297
8,► Suburbs of Invercargill‎ (13 P),-36.871782,174.638724
9,"► Suburbs of Napier, New Zealand‎ (16 P)",-36.846657,174.630150


In [46]:
kl_df.to_csv("kl_df.csv", index=False)

In [48]:
address = 'Suburbs of Auckland, New Zealand'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Auckland, New Zealand {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Auckland, New Zealand -36.8554342, 174.8456128.


In [49]:
# create map of Auckland, New Zealand using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [50]:
map_kl.save('map_kl.html')

In [51]:
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


In [56]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [54]:
def get_google_auth(state=None, token=None):
    if token:
        return OAuth2Session(Auth.CLIENT_ID, token=token)
    if state:
        return OAuth2Session(
            Auth.CLIENT_ID,
            state=state,
            redirect_uri=Auth.REDIRECT_URI,
            scope=['email']
        )
    oauth = OAuth2Session(
        Auth.CLIENT_ID,
        redirect_uri=Auth.REDIRECT_URI,
        scope=['email']
    )
    return oauth

In [55]:
CLIENT_ID = '0MJA3NYYG3U2ZY1LTZN2OYEHS3Y3WVSON2GBSO3IL4EDYVIR' # your Foursquare ID
CLIENT_SECRET = 'WGWSAF2TKVUQPE3PD0N3EOITFVBY5EYP1VCZI3BMUG0ROUS5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [57]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(979, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,► Suburbs of Auckland‎ (208 P),-36.709183,174.733092,Sherwood Park,-36.717437,174.735803,Park
1,► Suburbs of Auckland‎ (208 P),-36.709183,174.733092,Deep Creek Brewing Company,-36.713687,174.748776,Brewery
2,► Suburbs of Auckland‎ (208 P),-36.709183,174.733092,Browns Bay Beach,-36.714291,174.750037,Beach
3,► Suburbs of Auckland‎ (208 P),-36.709183,174.733092,Waiake Beach,-36.702890,174.751484,Beach
4,► Suburbs of Auckland‎ (208 P),-36.709183,174.733092,Hatz Donburi,-36.713914,174.747985,Japanese Restaurant


In [58]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
► Suburbs of Auckland‎ (208 P),29,29,29,29,29,29
"► Suburbs of Blenheim, New Zealand‎ (5 P)",36,36,36,36,36,36
"► Suburbs of Christchurch‎ (2 C, 73 P)",100,100,100,100,100,100
"► Suburbs of Dunedin‎ (2 C, 49 P)",100,100,100,100,100,100
"► Suburbs of Gisborne, New Zealand‎ (11 P)",47,47,47,47,47,47
► Suburbs of Greymouth‎ (3 P),100,100,100,100,100,100
"► Suburbs of Hamilton, New Zealand‎ (50 P)",100,100,100,100,100,100
"► Suburbs of Hastings, New Zealand‎ (11 P)",47,47,47,47,47,47
► Suburbs of Invercargill‎ (13 P),41,41,41,41,41,41


In [59]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 138 uniques categories.


In [60]:
venues_df['VenueCategory'].unique()[:50]

array(['Park', 'Brewery', 'Beach', 'Japanese Restaurant', 'Café',
       'Neighborhood', 'Fish & Chips Shop', 'Pizza Place', 'Bistro',
       'Indian Restaurant', 'Noodle House', 'Coffee Shop',
       'Ice Cream Shop', 'Supermarket', 'Asian Restaurant',
       'Grocery Store', 'Restaurant', 'Food', 'Soccer Field', 'Bakery',
       'Golf Course', 'Chinese Restaurant', 'Fruit & Vegetable Store',
       'Movie Theater', 'Sports Bar', 'Miscellaneous Shop',
       'Tapas Restaurant', 'Bar', 'Cantonese Restaurant',
       'Fast Food Restaurant', 'Bookstore', 'Shopping Mall',
       'Burger Joint', 'Theater', 'Garden Center', 'Monument / Landmark',
       'Hotel', 'Record Shop', 'Tennis Stadium', 'Art Gallery',
       'Dessert Shop', 'City Hall', 'Food Court', 'Sushi Restaurant',
       'Korean Restaurant', 'Brazilian Restaurant',
       'South Indian Restaurant', 'Steakhouse', 'Buffet', 'Gym'],
      dtype=object)

In [61]:
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [62]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(979, 139)


,Neighborhoods,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Beach,Beer Bar,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Café,Candy Store,Cantonese Restaurant,Chinese Restaurant,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Malay Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,Music Store,Neighborhood,Noodle House,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skate Park,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Speakeasy,Sports Bar,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volcano,Water Park,Wine Bar
0,► Suburbs of Auckland‎ (208 P),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,► Suburbs of Auckland‎ (208 P),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,► Suburbs of Auckland‎ (208 P),0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,► Suburbs of Auckland‎ (208 P),0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,► Suburbs of Auckland‎ (208 P),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(19, 139)


,Neighborhoods,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Beach,Beer Bar,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Café,Candy Store,Cantonese Restaurant,Chinese Restaurant,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Malay Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,Music Store,Neighborhood,Noodle House,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skate Park,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Speakeasy,Sports Bar,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volcano,Water Park,Wine Bar
0,► Suburbs of Auckland‎ (208 P),0.00000,0.00,0.00000,0.034483,0.000000,0.00,0.000000,0.00,0.034483,0.000000,0.000000,0.068966,0.00,0.034483,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00000,0.034483,0.00,0.00,0.000000,0.103448,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.068966,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.034483,0.00,0.034483,0.00000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.034483,0.000000,0.034483,0.000000,0.00,0.000000,0.000000,0.000000,0.034483,0.068966,0.00,0.000000,0.068966,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.068966,0.034483,0.00000,0.034483,0.000000,0.00000,0.068966,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.034483,0.000000,0.000000,0.00,0.000000,0.00,0.034483,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.034483,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00000,0.00
1,"► Suburbs of Blenheim, New Zealand‎ (5 P)",0.00000,0.00,0.00000,0.055556,0.000000,0.00,0.000000,0.00,0.055556,0.027778,0.000000,0.000000,0.00,0.000000,0.00,0.027778,0.00,0.000000,0.000000,0.00,0.00000,0.000000,0.00,0.00,0.027778,0.083333,0.000000,0.027778,0.194444,0.00,0.00,0.00,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.00,0.055556,0.00,0.000000,0.00000,0.00,0.00,0.000000,0.000000,0.000000,0.027778,0.00000,0.000000,0.027778,0.00000,0.000000,0.000000,0.000000,0.055556,0.000000,0.083333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.027778,0.00,0.00,0.000000,0.00,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [64]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

11

In [65]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [66]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,► Suburbs of Auckland‎ (208 P),0.000000
1,"► Suburbs of Blenheim, New Zealand‎ (5 P)",0.027778
2,"► Suburbs of Christchurch‎ (2 C, 73 P)",0.010000
3,"► Suburbs of Dunedin‎ (2 C, 49 P)",0.020000
4,"► Suburbs of Gisborne, New Zealand‎ (11 P)",0.021277


In [67]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 2, 0, 0, 2, 2, 0, 0, 1], dtype=int32)

In [68]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [69]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,► Suburbs of Auckland‎ (208 P),0.000000,1
1,"► Suburbs of Blenheim, New Zealand‎ (5 P)",0.027778,0
2,"► Suburbs of Christchurch‎ (2 C, 73 P)",0.010000,2
3,"► Suburbs of Dunedin‎ (2 C, 49 P)",0.020000,0
4,"► Suburbs of Gisborne, New Zealand‎ (11 P)",0.021277,0


In [70]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(19, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,► Suburbs of Auckland‎ (208 P),0.000000,1,-36.709183,174.733092
1,"► Suburbs of Blenheim, New Zealand‎ (5 P)",0.027778,0,-36.910492,174.926942
2,"► Suburbs of Christchurch‎ (2 C, 73 P)",0.010000,2,-36.857603,174.768211
3,"► Suburbs of Dunedin‎ (2 C, 49 P)",0.020000,0,-36.844479,174.748203
4,"► Suburbs of Gisborne, New Zealand‎ (11 P)",0.021277,0,-36.916980,174.702297


In [71]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(19, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
18,► Suburbs of Whangarei‎ (13 P),0.024390,0,-36.871782,174.638724
1,"► Suburbs of Blenheim, New Zealand‎ (5 P)",0.027778,0,-36.910492,174.926942
3,"► Suburbs of Dunedin‎ (2 C, 49 P)",0.020000,0,-36.844479,174.748203
4,"► Suburbs of Gisborne, New Zealand‎ (11 P)",0.021277,0,-36.916980,174.702297
14,► Suburbs of Rotorua‎ (5 P),0.027778,0,-36.910492,174.926942
7,"► Suburbs of Hastings, New Zealand‎ (11 P)",0.021277,0,-36.916980,174.702297
8,► Suburbs of Invercargill‎ (13 P),0.024390,0,-36.871782,174.638724
12,► Suburbs of Palmerston North‎ (22 P),0.019608,0,-40.349208,175.611964
16,► Suburbs of Tauranga‎ (13 P),0.000000,1,-36.906813,174.546949
15,► Suburbs of Taupo‎ (6 P),0.000000,1,-36.926109,174.681216


In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [74]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [75]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
18,► Suburbs of Whangarei‎ (13 P),0.024390,0,-36.871782,174.638724
1,"► Suburbs of Blenheim, New Zealand‎ (5 P)",0.027778,0,-36.910492,174.926942
3,"► Suburbs of Dunedin‎ (2 C, 49 P)",0.020000,0,-36.844479,174.748203
4,"► Suburbs of Gisborne, New Zealand‎ (11 P)",0.021277,0,-36.916980,174.702297
14,► Suburbs of Rotorua‎ (5 P),0.027778,0,-36.910492,174.926942
7,"► Suburbs of Hastings, New Zealand‎ (11 P)",0.021277,0,-36.916980,174.702297
8,► Suburbs of Invercargill‎ (13 P),0.024390,0,-36.871782,174.638724
12,► Suburbs of Palmerston North‎ (22 P),0.019608,0,-40.349208,175.611964


In [76]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
16,► Suburbs of Tauranga‎ (13 P),0.0,1,-36.906813,174.546949
15,► Suburbs of Taupo‎ (6 P),0.0,1,-36.926109,174.681216
13,"► Suburbs of Queenstown, New Zealand‎ (9 P)",0.0,1,-36.890802,174.764891
0,► Suburbs of Auckland‎ (208 P),0.0,1,-36.709183,174.733092
10,"► Suburbs of Nelson, New Zealand‎ (30 P)",0.0,1,-36.908926,174.917732
17,► Suburbs of Wellington‎ (4 C),0.0,1,-36.893583,174.930080
11,► Suburbs of New Plymouth‎ (14 P),0.0,1,-36.812978,174.786833
9,"► Suburbs of Napier, New Zealand‎ (16 P)",0.0,1,-36.846657,174.630150


In [77]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
6,"► Suburbs of Hamilton, New Zealand‎ (50 P)",0.01,2,-36.848399,174.764388
5,► Suburbs of Greymouth‎ (3 P),0.01,2,-36.848399,174.764388
2,"► Suburbs of Christchurch‎ (2 C, 73 P)",0.01,2,-36.857603,174.768211


Most of the shopping malls are concentrated in the central area of Auckland with cluster 0 having many malls in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls.